<a href="https://colab.research.google.com/github/BryceRodgers7/CoTorch/blob/main/DuneMovieCorpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# DUNE (2021) MOVIE SCRIPT ML PROJECT (SENTIMENT ANALYSIS?)
# STEP 1: DOWNLOAD DATASET

# download dataset (pdf of script)
import requests

def download_pdf_from_url(url, save_path):
    response = requests.get(url)
    with open(save_path, 'wb') as file:
        file.write(response.content)

# URL of the PDF file
pdf_url = "https://d2bu9v0mnky9ur.cloudfront.net/academy2021/scripts/duneMxFtT98NYwBsMltl20211109/dune_final_shooting_script_6_19_20.pdf"

# Path to save the downloaded PDF file
save_path = "dune_script.pdf"

# Download the PDF file
download_pdf_from_url(pdf_url, save_path)

print("PDF file downloaded successfully.")


PDF file downloaded successfully.


In [2]:
pip install PyPDF2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.7 MB/s eta 0:00:00


In [ ]:
pip install BERTopic

In [4]:
# STEP 2: EXTRACT TEXT FROM PDF
import PyPDF2
import re
import os

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

# helper function to list all unique strings
def find_unique_strings(text, pattern):
    # Find all matches of the pattern in the text
    matches = re.findall(pattern, text)

    # Use a set to store unique matches
    unique_matches = set(matches)

    return unique_matches

# helper function to print first few lines
def print_first_x_lines(text, x):
    lines = text.split('\n')
    for i in range(min(x, len(lines))):
        print(lines[i])

# Path to your PDF file
pdf_path = "dune_script.pdf"

# Extract text from PDF
text = extract_text_from_pdf(pdf_path)

# print(text)

In [6]:
# STEP 3: PURGE FORMATTING-TEXT FROM CORPUS
# remove title page from dataset
page_0 = "DUNE\nScreenplay by\nJON SPAIHTS and DENIS VILLENEUVE \nand ERIC ROTH  \nBased on the novel  'Dune' by FRANK HERBERT  \nJune 19, 2020  Salmon \nRevisions\nFinal Shooting DraftTITLES ON BLACK, APPEARING ONE AFTER THE OTHER *\nPrologue about SPICE TBD *\nFADE IN: *\nDREAM SEQUENCE: EXT. ARRAKIS - END OF DAY 1 1"
preprocessed_text = text.replace(page_0, "")

# remove revision-notes from dataset
pattern = r'Salmon Rev\. \(\d{2}\/\d{2}\/\d{4}\)'
preprocessed_text = re.sub(pattern, "", preprocessed_text)
pattern = r'Salmon Rev\. \(\d{2}\/\d{2}\/\d{2}\)'
preprocessed_text = re.sub(pattern, "", preprocessed_text)

pattern = r'Buff Rev\. \(\d{2}\/\d{2}\/\d{4}\)'
preprocessed_text = re.sub(pattern, "", preprocessed_text)
pattern = r'Buff Rev\. \(\d{2}\/\d{2}\/\d{2}\)'
preprocessed_text = re.sub(pattern, "", preprocessed_text)

pattern = r'Golden Rev\. \(\d{2}\/\d{2}\/\d{4}\)'
preprocessed_text = re.sub(pattern, "", preprocessed_text)
pattern = r'Golden Rev\. \(\d{2}\/\d{2}\/\d{2}\)'
preprocessed_text = re.sub(pattern, "", preprocessed_text)

# remove the following from the dataset:
preprocessed_text = preprocessed_text.replace("FADE TO BLACK", "")
preprocessed_text = preprocessed_text.replace("THE END", "")
preprocessed_text = preprocessed_text.replace("(MORE)", "")



In [18]:
# STEP 5: DIVIDE CORPUS INTO DOCUMENTS
# now we should cut up the text into documents in different ways to create different structured data as input for different models so we can compare the models
#   possible groupings:
#   1 document = 1 FULL scene  (all dialogue, all scene description)
#   1 document = 1 sub-scene  (all dialogue, all scene description)
#   1 document = 1 script page (all dialogue, all scene description)
#   repeat the above for dialogue-only or scene-description only??

# remove page numbers (replace with a PAGEPAGE marker to denote a new document)
dune_docs = []
dune_docs.append("page0 empty..")
remaining_text = preprocessed_text

for x in range(2, 111):
  # print(f'\n\n\n************************\nITERATION/PAGE {x-1}:')
  script_page_num_to_remove = f'{x}.'
  page_removed_text = remaining_text.replace(script_page_num_to_remove, "xyz123", 1)

  # ensure a page number was found & removed
  if (remaining_text == page_removed_text):
    print(f"ALERT SCRIPT PAGE NUMBER STRING {script_page_num_to_remove} was not found!!! Investigate..")
    dune_docs.append(f"EMPTY PAGE NUMBER {x-1}")
  else:
    two_docs = page_removed_text.split('xyz123')

    # make sure replace created only 2 pages
    if (len(two_docs) >= 3):
        print(f'lmao how did this happen?? on iteration {x-1}')

    page_doc = two_docs[0]
    current_page_num_to_remove = f'{x-1}.'
    page_doc = page_doc.replace(current_page_num_to_remove, "", 1)
    # print(f'page number {x-1} is \n{page_doc}')
    dune_docs.append(page_doc)

    if (x == 110):
      # create last page
      page_doc = two_docs[1]
      page_doc = page_doc.replace("110.", "", 2)
      # print(f'page number 110 (last page) is \n{page_doc}')
      dune_docs.append(page_doc)
      # if(len(two_docs[1]) > 50):
      #   print(f'\nrest of the corpus...')
      #   print_first_x_lines(two_docs[1], 10)
      #   print(f'and so on...')
      # else:
      #   print(f'doc1 is \n{two_docs[1]}')

    remaining_text = two_docs[1]

for i, doc in enumerate(dune_docs):
  if (int(i) < 10):
    print(f"doc {i} is {doc}")

# print("pagenumber replacement complete (if no output before this then all page numbers were replaced successfully).")

# ALL SCENE CODES IN ORDER (note the first scene code is not in corpus, and some scene codes are repeated.)
scence_codes = ['0','2A','2C','2D','2B','1A','1C','1D','1E','1F','1G','22','3A','23A','22aaB','22aaA','14','19','20A','21A','21B','5','6','7','9','9A','10','11','11A','22aA','22A','24','25','26','27','28','28B','29','30','33','38','35','34','37','37A','38aA','42AC','42AD','47BB','47B','47BA','47C','42BB','42BC','42B','44','45','46','47','48A','48','51','52','53','55A','55','55AAB','55AAA','55aB','55aA','58','61','58A','60','64','64A','64B','65','67','68','69','71','72','73','74','74A','75','76','66','66A','80','78','78A','78B','86','87','81','93','88','89','90','91','94','95','96','95A','95B','95C','95E','95B','98','99','100','101','102','102A','103','104','105','105A','106','107','108','108A','109','111','112','113','114','125','126','127','128','128A','128B','129','131','132','132A','132B','132C','133']


# test code to find all scene codes
# unique_strings = find_unique_strings(preprocessed_text, pattern)
# for strang in unique_strings:
#   print(strang, end=",")
#   # good list, get rid of them all

# remove scene codes TODO: replace with SCENESCENE marker later to denote a new document
# pattern = r'\ \d+[A-Z]'
# preprocessed_text = re.sub(pattern, " ", preprocessed_text)

# # dune_docs = preprocessed_text.split("PAGEPAGE")

# remaining_text = preprocessed_text


# split each page into its own document, noting any skipped pages
# docs = []
# for x in range(2, 110):
#   page_marker = f"PAGE{x}PAGE"
#   two_docs = remaining_text.split(page_marker)
#   if (two_docs[0] == ""):
#     print(f"two_docs[0] is empty for page marker {page_marker}")
#   elif (len(two_docs) == 1):
#     print(f"no split occurred for {page_marker} forcing x to increment...")
#     continue
#   elif (two_docs[1] == ""):
#     print(f"two_docs[0] is empty for page marker {page_marker} (rly weird)")
#   else:
#     docs.append(two_docs[0])
#     remaining_text = two_docs[1]

# print(f'len(docs) is {len(docs)}')

# for i in range (0, 110):
#   if (0 <= i) and (i < len(docs)):
#     print(f'PAGE NUMBER {i} is:\n {docs[i]}')
#   else:
#     print(f'no page number {i}')

doc 0 is page0 empty..
doc 1 is 
The planet Arrakis, as seen from space.
Track across its endless windswept terrain.We glide into a low-hanging dark cloud that’s generated by a 
massive mining vehicle, a HARVESTER, kicking up glowing flecks of SPICE. We PUSH through the SPICE, creating a dreamlike swirl of orange flakes.
Through the swirl WE REVEAL a SECOND HARVESTER airborne, 
being hauled by a powerful CARRYALL.
ON THE GROUND - HARKONNEN SOLDIERS flanking the harvester, 
leading the industrial nightmare through the darkness. One of them holds a massive flag bearing the HARKONNEN EMBLEM.
Now these soldiers are observed through the P.O.V. of a 
thermal scope.  Reveal that this scope is attached to a 
strange MISSLE LAUNCHER, one of multiple cloth-shrouded weapons being wielded by a small band of blue-eye FREMEN FIGHTERS taking cover behind a sprawling black rock. A young female fighter, CHANI, is among them; along with a man who we will know later as JAMIS. A closer look at Chani.   
F

In [10]:
# STEP 6: PREPROCESS DOCUMENTS - stem, stop, tokenize


# import nltk
# from nltk.stem import PorterStemmer
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# import string

# stemmer = PorterStemmer()
# stop = set(stopwords.words('english') + list(string.punctuation))

# def tokenize_corpus(corpus):
#     tokenized_corpus = []
#     for document in corpus:
#         tokens = word_tokenize(document)
#         tokenized_corpus.append(tokens)
#     return tokenized_corpus

# def stem_corpus(corpus):
#     stemmed_corpus = []
#     for document in corpus:
#         words = word_tokenize(document)
#         stemmed_words = [stemmer.stem(word) for word in words]
#         stemmed_document = ' '.join(stemmed_words)
#         stemmed_corpus.append(stemmed_document)
#     return stemmed_corpus

# def remove_stop_words(corpus):
#     cleaned_corpus = []
#     for document in corpus:
#         words = word_tokenize(document)
#         cleaned_words = [word for word in words if word.lower() not in stop]
#         cleaned_document = ' '.join(cleaned_words)
#         cleaned_corpus.append(cleaned_document)
#     return cleaned_corpus

# # Tokenize the corpus
# tokenized_corpus = tokenize_corpus(dune_docs)

# cleaned_corpus = remove_stop_words(corpus)
# stemmed_corpus = stem_corpus(dune_docs)

# [i for i in word_tokenize(sent.lower()) if i not in stop]

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import unicodedata


nltk.download('punkt')
nltk.download('stopwords')
print(string.punctuation)

# Initialize the Porter stemmer and get the list of English stop words
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def fix_periods(document):
  return document.replace('.', ' ')

def remove_punctuations(document):
  punctuations = string.punctuation
  return document.translate(str.maketrans('', '', punctuations))

def remove_stopwords(document):
  return " ".join([word for word in document.split() if word not in stop_words])

def remove_special_characters(document):
  document = re.sub('[^a-zA-Z0-9]', ' ', document)
  # limit spaces to 1
  document = re.sub('\s+', ' ', document)
  return document

def stem_words(document):
  return " ".join([stemmer.stem(word) for word in document.split()])

def preprocess_document(document):
    document = document.lower()
    document = remove_punctuations(document)
    document = remove_stopwords(document)
    document = remove_special_characters(document)
    processed_document = stem_words(document)
    # add in lemmatizing later?
    return processed_document

def preprocess_corpus(corpus):
    preprocessed_corpus = []
    for document in corpus:
        preprocessed_document = preprocess_document(document)
        preprocessed_corpus.append(preprocessed_document)
    return preprocessed_corpus


# Preprocess the corpus
preprocessed_corpus = preprocess_corpus(dune_docs)

# Print the preprocessed corpus
for i, document in enumerate(preprocessed_corpus):
    print(f"Document {i+1}: {document}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
Document 1: page0 empty..
Document 2: planet arrakis, seen space. track across endless windswept terrain.w glide low-hang dark cloud that’ gener massiv mine vehicle, harvester, kick glow fleck spice. push spice, creat dreamlik swirl orang flakes. swirl reveal second harvest airborne, haul power carryall. ground - harkonnen soldier flank harvester, lead industri nightmar darkness. one hold massiv flag bear harkonnen emblem. soldier observ p.o.v. thermal scope. reveal scope attach strang missl launcher, one multipl cloth-shroud weapon wield small band blue-ey fremen fighter take cover behind sprawl black rock. young femal fighter, chani, among them; along man know later jamis. closer look chani. flicker fremen plasma laser lanc second crawler, explod carryal carri it. rocket launch harvest retaliate, inciner sever fremen brutal strike. surviv fremen run cover crevass rocki position. last open chani. disappear underground opening, chani turn look back soul

In [ ]:
# STEP 7: CREATE MODEL
from bertopic import BERTopic

topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2")
topics, probs = topic_model.fit_transform(preprocessed_corpus)

topic_model.get_topic_info()


# WOWWWWW these topics aren't bad at all

,Topic,Count,Name,Representation,Representative_Docs
0,-1,11,-1_trooper_duncan_paul_first,"[trooper, duncan, paul, first, ornithopt, deaf...",[first trooper unlatch jessica s seat har yank...
1,0,68,0_paul_leto_gurney_harkonnen,"[paul, leto, gurney, harkonnen, kyne, sand, id...",[leto mani men crawler kyne crew twentyon leto...
2,1,32,1_paul_jessica_mother_reverend,"[paul, jessica, mother, reverend, mohiam, jami...",[take hand box boy look reluctantli paul compl...


from matplotlib import pyplot as plt
_df_10['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_11['Topic'].plot(kind='hist', bins=20, title='Topic')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_12['Count'].plot(kind='hist', bins=20, title='Count')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_13.groupby('Name').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_21['Topic'].plot(kind='line', figsize=(8, 4), title='Topic')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_22['Count'].plot(kind='line', figsize=(8, 4), title='Count')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_23['Name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_23, x='index', y='Name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_24['Name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_24, x='Topic', y='Name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_25['Name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_25, x='Count', y='Name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
# regex for removing rev's:
# (Salmon Rev\.|Buff Ref\.) \(\d{2}\/\d{2}\/\d{2}\)

# regex for removing page number lines:
# ^\d{1,3}[a-zA-Z]{1,3} .+ [a-zA-Z]{1,3}\d{1,3}$

import re

pattern = r'^\d{1,3}[a-zA-Z]{1,3} .+ [a-zA-Z]{1,3}\d{1,3}$'

line1 = "123abc This is a test line abc123"
line2 = "this is a regular line of dialogue"
if re.match(pattern, line1):
    print("LINE1 MATCH FOUND")
else:
    print("LINE1 No match")

if re.match(pattern, line2):
    print("LINE2 MATCH FOUND")
else:
    print("LINE2 No match")





LINE1 MATCH FOUND
LINE2 No match


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 34.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 21.6 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━